In [1]:
%%time
import numpy as np
# import lightgbm as lgb

from functions import *
from datetime import datetime
# from sklearn.model_selection import train_test_split

np.random.seed(0)

first_test_day = 1914
sales, prices, calendar, summary, encoder = prepare_data(first_test_day)

CPU times: user 19 s, sys: 1.86 s, total: 20.9 s
Wall time: 22.3 s


In [ ]:
# calculate disagg coefficients

In [2]:
# clean calendar from useless columns
calendar.drop(['year', 'd','event_name_1','event_name_2','event_type_1','event_type_2','snap_CA','snap_TX','snap_WI','weekday' ], axis=1, inplace=True)
calendar.set_index('date', inplace=True) # Saturday -1, Friday -7
calendar['month'] = calendar['month'].astype('uint8')
calendar['wday'] = calendar['wday'].astype('uint8')
calendar.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1969 entries, 2011-01-29 to 2016-06-19
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   wm_yr_wk  1969 non-null   int64
 1   wday      1969 non-null   uint8
 2   month     1969 non-null   uint8
dtypes: int64(1), uint8(2)
memory usage: 34.6 KB


In [4]:
# remove zeroes before first sale from history

firstbuy = sales[sales['qty']!=0].groupby('id')[['date']].min()
firstbuy.rename(columns={'date':'firstbuy_date'}, inplace=True)
# if 'firstbuy_date' not in summary.columns:
#     summary = summary.join(firstbuy, on='id', how='left')
if 'firstbuy_date' not in sales.columns:
    sales = sales.join(firstbuy, on='id', how='left')
del firstbuy
print(sales.shape)
sales = sales[sales['date'] >= sales['firstbuy_date']]
sales.drop('firstbuy_date', axis=1, inplace=True)
print(sales.shape)

sales = sales.join(calendar, on='date')
sales.drop(['id', 'date'], axis=1, inplace=True)
sales

(59181090, 4)
(46796220, 3)


,qty,wm_yr_wk,wday,month
7,12,11101,1,1
8,2,11101,1,1
14,4,11101,1,1
15,5,11101,1,1
21,2,11101,1,1
...,...,...,...,...
59181085,0,11617,2,5
59181086,0,11617,2,5
59181087,0,11617,2,5
59181088,0,11617,2,5


In [5]:
df_wday = sales[['month', 'wday', 'qty']].groupby(['month', 'wday']).sum()[['qty']]
df_sum = sales[['month', 'qty']].groupby(['month']).sum()[['qty']]
df_wday.reset_index(inplace=True)
df_wday = df_wday.join(df_sum, on='month', rsuffix='sum')
df_wday['ratio'] = df_wday['qty'] / df_wday['qtysum']
import matplotlib.pyplot as plt
df_wday.pivot(index='wday', columns='month', values='ratio')

month,1,2,3,4,5,6,7,8,9,10,11,12
wday,,,,,,,,,,,,
1,0.186667,0.176874,0.178002,0.175237,0.170551,0.171869,0.151886,0.176685,0.170161,0.174089,0.176326,0.169618
2,0.173850,0.171873,0.180465,0.174601,0.162074,0.172401,0.162504,0.172915,0.179059,0.165229,0.181053,0.160158
3,0.130882,0.139838,0.132352,0.132832,0.132536,0.143761,0.134762,0.134309,0.143782,0.133599,0.133548,0.147314
4,0.117612,0.122619,0.126394,0.127742,0.126117,0.125945,0.135803,0.118213,0.129358,0.125249,0.124987,0.134970
5,0.117710,0.127294,0.119458,0.125475,0.124701,0.121377,0.135262,0.122488,0.120389,0.128579,0.126824,0.127675
6,0.126371,0.124792,0.126191,0.120464,0.132409,0.122502,0.132675,0.125278,0.120204,0.130228,0.115905,0.129429
7,0.146908,0.136710,0.137138,0.143649,0.151612,0.142145,0.147109,0.150113,0.137046,0.143027,0.141358,0.130835


In [10]:
df_wday[[ 'month', 'wday', 'ratio']].to_csv('./work/disagg_coefs.csv', index=False)

In [25]:
pred = pd.read_csv('submission_2020-04-24 18:58:53.csv')
pred = pd.melt(pred, id_vars=['id'], value_vars=pred.columns[1:])
pred

,id,variable,value
0,HOBBIES_1_001_CA_1_validation,F1,1.004219
1,HOBBIES_1_002_CA_1_validation,F1,0.290391
2,HOBBIES_1_003_CA_1_validation,F1,0.357772
3,HOBBIES_1_004_CA_1_validation,F1,1.971706
4,HOBBIES_1_005_CA_1_validation,F1,1.129227
...,...,...,...
1707435,FOODS_3_823_WI_3_evaluation,F28,0.000000
1707436,FOODS_3_824_WI_3_evaluation,F28,0.000000
1707437,FOODS_3_825_WI_3_evaluation,F28,0.000000
1707438,FOODS_3_826_WI_3_evaluation,F28,0.000000


In [26]:
flist = [f'F{i}' for i in range(1,29)]
dlist = [f'd_{i}' for i in range(1914,1942)]
mapping = dict(zip(flist,dlist))
pred['d'] = pred['variable'].map(mapping)
pred

,id,variable,value,d
0,HOBBIES_1_001_CA_1_validation,F1,1.004219,d_1914
1,HOBBIES_1_002_CA_1_validation,F1,0.290391,d_1914
2,HOBBIES_1_003_CA_1_validation,F1,0.357772,d_1914
3,HOBBIES_1_004_CA_1_validation,F1,1.971706,d_1914
4,HOBBIES_1_005_CA_1_validation,F1,1.129227,d_1914
...,...,...,...,...
1707435,FOODS_3_823_WI_3_evaluation,F28,0.000000,d_1941
1707436,FOODS_3_824_WI_3_evaluation,F28,0.000000,d_1941
1707437,FOODS_3_825_WI_3_evaluation,F28,0.000000,d_1941
1707438,FOODS_3_826_WI_3_evaluation,F28,0.000000,d_1941


In [27]:
calendar = read_calendar()
calendar = calendar.loc[1913:1940, ['wday', 'month','d']]

In [28]:
pred = pred.join(calendar.set_index('d'), on='d')
pred

,id,variable,value,d,wday,month
0,HOBBIES_1_001_CA_1_validation,F1,1.004219,d_1914,3,4
1,HOBBIES_1_002_CA_1_validation,F1,0.290391,d_1914,3,4
2,HOBBIES_1_003_CA_1_validation,F1,0.357772,d_1914,3,4
3,HOBBIES_1_004_CA_1_validation,F1,1.971706,d_1914,3,4
4,HOBBIES_1_005_CA_1_validation,F1,1.129227,d_1914,3,4
...,...,...,...,...,...,...
1707435,FOODS_3_823_WI_3_evaluation,F28,0.000000,d_1941,2,5
1707436,FOODS_3_824_WI_3_evaluation,F28,0.000000,d_1941,2,5
1707437,FOODS_3_825_WI_3_evaluation,F28,0.000000,d_1941,2,5
1707438,FOODS_3_826_WI_3_evaluation,F28,0.000000,d_1941,2,5


In [29]:
coefs = pd.read_csv('./work/disagg_coefs.csv')
coefs

,month,wday,ratio
0,1,1,0.186667
1,1,2,0.173850
2,1,3,0.130882
3,1,4,0.117612
4,1,5,0.117710
...,...,...,...
79,12,3,0.147314
80,12,4,0.134970
81,12,5,0.127675
82,12,6,0.129429


In [30]:
pred = pred.join(coefs.set_index(['month','wday']), on=['month','wday'])
pred

,id,variable,value,d,wday,month,ratio
0,HOBBIES_1_001_CA_1_validation,F1,1.004219,d_1914,3,4,0.132832
1,HOBBIES_1_002_CA_1_validation,F1,0.290391,d_1914,3,4,0.132832
2,HOBBIES_1_003_CA_1_validation,F1,0.357772,d_1914,3,4,0.132832
3,HOBBIES_1_004_CA_1_validation,F1,1.971706,d_1914,3,4,0.132832
4,HOBBIES_1_005_CA_1_validation,F1,1.129227,d_1914,3,4,0.132832
...,...,...,...,...,...,...,...
1707435,FOODS_3_823_WI_3_evaluation,F28,0.000000,d_1941,2,5,0.162074
1707436,FOODS_3_824_WI_3_evaluation,F28,0.000000,d_1941,2,5,0.162074
1707437,FOODS_3_825_WI_3_evaluation,F28,0.000000,d_1941,2,5,0.162074
1707438,FOODS_3_826_WI_3_evaluation,F28,0.000000,d_1941,2,5,0.162074


In [31]:
pred['prediction'] = pred['value'] * 7 * pred['ratio']
pred

,id,variable,value,d,wday,month,ratio,prediction
0,HOBBIES_1_001_CA_1_validation,F1,1.004219,d_1914,3,4,0.132832,0.933749
1,HOBBIES_1_002_CA_1_validation,F1,0.290391,d_1914,3,4,0.132832,0.270014
2,HOBBIES_1_003_CA_1_validation,F1,0.357772,d_1914,3,4,0.132832,0.332666
3,HOBBIES_1_004_CA_1_validation,F1,1.971706,d_1914,3,4,0.132832,1.833344
4,HOBBIES_1_005_CA_1_validation,F1,1.129227,d_1914,3,4,0.132832,1.049985
...,...,...,...,...,...,...,...,...
1707435,FOODS_3_823_WI_3_evaluation,F28,0.000000,d_1941,2,5,0.162074,0.000000
1707436,FOODS_3_824_WI_3_evaluation,F28,0.000000,d_1941,2,5,0.162074,0.000000
1707437,FOODS_3_825_WI_3_evaluation,F28,0.000000,d_1941,2,5,0.162074,0.000000
1707438,FOODS_3_826_WI_3_evaluation,F28,0.000000,d_1941,2,5,0.162074,0.000000


In [32]:
pred = pred[['id', 'variable', 'prediction']]
pred

,id,variable,prediction
0,HOBBIES_1_001_CA_1_validation,F1,0.933749
1,HOBBIES_1_002_CA_1_validation,F1,0.270014
2,HOBBIES_1_003_CA_1_validation,F1,0.332666
3,HOBBIES_1_004_CA_1_validation,F1,1.833344
4,HOBBIES_1_005_CA_1_validation,F1,1.049985
...,...,...,...
1707435,FOODS_3_823_WI_3_evaluation,F28,0.000000
1707436,FOODS_3_824_WI_3_evaluation,F28,0.000000
1707437,FOODS_3_825_WI_3_evaluation,F28,0.000000
1707438,FOODS_3_826_WI_3_evaluation,F28,0.000000


In [35]:
pred.pivot(index='id', columns='variable',values='prediction').sort_values('id').to_csv('./submission_2020-04-24 21:10:00')